In [ ]:
#======================================================================
#
#     This routine solves an infinite horizon growth model 
#     with dynamic programming and sparse grids
#
#     The model is described in Scheidegger & Bilionis (2017)
#     https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2927400
#
#     external libraries needed:
#     - IPOPT (https://projects.coin-or.org/Ipopt)
#     - PYIPOPT (https://github.com/xuy/pyipopt)
#     - TASMANIAN (http://tasmanian.ornl.gov/)
#
#     Simon Scheidegger, 11/16 ; 07/17
#======================================================================

import nonlinear_solver_initial as solver     #solves opt. problems for terminal VF
import nonlinear_solver_iterate as solviter   #solves opt. problems during VFI
from parameters import *                      #parameters of model
import interpolation as interpol              #interface to sparse grid library/terminal VF
import interpolation_iter as interpol_iter    #interface to sparse grid library/iteration
import postprocessing as post                 #computes the L2 and Linfinity error of the model

import TasmanianSG                            #sparse grid library
import numpy as np


#======================================================================
# Start with Value Function Iteration

# terminal value function : "ss", no uncertainty --> ss values of c,l,k
valnew=TasmanianSG.TasmanianSparseGrid()
lvalnew=[valnew, valnew, valnew, valnew, valnew]
if (numstart==0):
    for g in range(5):
        iG = g
        print("Evaluating grid for state", iG)
        lvalnew[iG]=interpol.sparse_grid(n_agents, iDepth, iG)
        #lvalnew[iG].plotResponse2D()
        points = lvalnew[iG].getPoints()
        #print(points)

    
# value function during iteration --> take ss values of c,l,k and 
else:
    print("Set numstart to 0 PLEEEEASE")
    
valold=TasmanianSG.TasmanianSparseGrid()
lvalold=[valold, valold, valold, valold, valold]

for i in range(numstart, numits):
    print("Iteration number", i) 
    for g in range(5):
        iG = g
        print("Evaluating grid for state", iG)
        lvalnew[iG]=TasmanianSG.TasmanianSparseGrid()
        lvalnew[iG]=interpol_iter.sparse_grid_iter(n_agents, iDepth, iG, lvalold)
        points = lvalnew[iG].getPoints()

        lvalold[iG]=TasmanianSG.TasmanianSparseGrid()
        lvalold[iG]=lvalnew[iG]


#======================================================================
print( "===============================================================")
print( " " )
print( " Computation of a growth model of dimension ", n_agents ," finished after ", numits, " steps")
print( " " )
print( "===============================================================")
#======================================================================

# compute errors   
avg_err=post.ls_error(n_agents, numstart, numits, No_samples)

#======================================================================
print( "===============================================================")
print( " ")
print( " Errors are computed -- see errors.txt")
print( " ")
print( "===============================================================")
#======================================================================